# Andy Kong Arcpy Lab 1

#### Importing arcy and setting up the workspace


In [2]:
# import the system to be able to use arcpy
import sys
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.5\\bin')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.5\\arcpy')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.5\\ArcToolbox\\Scripts')
import arcpy

In [3]:
arcpy.env.overwriteOutput = True

arcpy.management.CopyFeatures(
    "C:\\Users\\andykong\\Documents\\Geog_458\\saep_bg10.shp",
    "C:\\Users\\andykong\\Documents\\Geog_458\\saep_bg10_copy.shp")


<Result 'C:\\Users\\andykong\\Documents\\Geog_458\\saep_bg10_copy.shp'>

In [14]:
import os
from subprocess import call
os.environ["GDAL_DATA"] = "C:\\OSGeo4W64\\share\\gdal"
os.environ["GDAL_DRIVER_PATH"] = "C:\\OSGeo4W64\\bin\\gdalplugins"
os.environ["PROJ_LIB"] = "C:\\OSGeo4W64\\share\\proj"
os.environ["PATH"] = "C:\\OSGeo4W64\\bin;"+os.environ["PATH"]+";C:\\OSGeo4W64\\apps\\msys\\bin;C:\\OSGeo4W64\\apps\\Python27\\Scripts"

In [6]:
arcpy.env.workspace = "C:\\Users\\andykong\\Documents\\Geog_458\\saep_bg10_copy"
input_shapefile = "C:\\Users\\andykong\\Documents\\Geog_458\\saep_bg10_copy.shp"
input_database = "C:\\Users\\andykong\\Documents\\Geog_458\\WashingtonFIPS.dbf"

#### Adding a county name field to the census shape file and joining the two county name with the FIPS code

In [7]:
arcpy.management.AddField(input_shapefile,"countyName","STRING")
arcpy.env.overwriteOutput = True


In [8]:
census_fields = arcpy.da.UpdateCursor(input_shapefile,["COUNTYFP10","countyName"])
    
for row1 in census_fields:
    Table_fields = arcpy.da.SearchCursor(input_database,["FIPSCounty","CountyName"])
    for row2 in Table_fields:
        if(row1[0]==row2[0]):
            row1[1]=row2[1]
    census_fields.updateRow(row1)
    del Table_fields
del census_fields


#### appending list of County names in to an array

In [9]:
county_list = []
cursor = arcpy.da.SearchCursor(input_database, ["CountyName"])
for row in cursor:
    county_list.append(row[0])
del cursor
print county_list

[u'Adams', u'Asotin', u'Benton', u'Chelan', u'Clallam', u'Clark', u'Columbia', u'Cowlitz', u'Douglas', u'Ferry', u'Franklin', u'Garfield', u'Grant', u'Grays Harbor', u'Island', u'Jefferson', u'King', u'Kitsap', u'Kittitas', u'Klickitat', u'Lewis', u'Lincoln', u'Mason', u'Okanogan', u'Pacific', u'Pend Oreille', u'Pierce', u'San Juan', u'Skagit', u'Skamania', u'Snohomish', u'Spokane', u'Stevens', u'Thurston', u'Wahkiakum', u'Walla Walla', u'Whatcom', u'Whitman', u'Yakima']


#### Converting list of County Shapefiles into GeoJson files

In [10]:
directory = "C:\\Users\\andykong\\Documents\\Geog_458\\Shapefiles"

In [12]:
for row in county_list:
    arcpy.FeatureClassToFeatureClass_conversion(input_shapefile,directory,str(row),'"CountyName" =' + "'%s'" %i)

In [15]:
for row in county_list:
    call(['C:\\OSGeo4W64\\bin\\ogr2ogr',   
      '-f','GeoJSON','-t_srs','WGS84',
      '-s_srs','EPSG:2927',         #followed lab instructions to Create EPSG from the proj of saep_bg10_copy
      'C:\\Users\\andykong\\Documents\\Geog_458\\Shapefiles' + row +'.geojson', #used i variable for each county
      'C:\\Users\\andykong\\Documents\\Geog_458\\Shapefiles' + row +'.shp'])

#### Creates a list of county population and rank the top ten largest population

In [25]:
#creates a list with the population of each county
pop_list = []
for i in county_list:
    pop = 0      
    cursor = arcpy.da.SearchCursor(input_shapefile, ["POP2017"], '"CountyName" = ' + "'%s'" %i) #where expression used from the above sql statement
    for row in cursor:
        pop += row[0]
    pop_list.append(pop) 

del cursor 
print pop_list


[19870.0, 22290.0, 193499.99999999997, 76829.99999999999, 74240.00000000001, 470999.9999999994, 4100.0, 105900.00000000001, 41420.0, 7740.0, 90330.00000000003, 2200.0, 95630.0, 72970.0, 82790.0, 31360.0, 2153700.0, 264300.00000000006, 44730.0, 21660.0, 77440.00000000003, 10700.0, 63190.00000000001, 42110.00000000001, 21249.999999999996, 13370.0, 859400.0000000003, 16510.0, 124100.00000000001, 11690.0, 789400.0000000005, 499800.0, 44510.0, 276899.9999999999, 4030.0, 61399.999999999985, 216300.00000000003, 48639.99999999999, 253000.0]


In [35]:
pop_list.sort(reverse=True) # organizes list in descending order
top = 0
while top < 10: #while statement for the top 10 populations
    print pop_list[top] 
    top += 1


2153700.0
859400.0
789400.0
499800.0
471000.0
276900.0
264300.0
253000.0
216300.0
193500.0


# Reflection

Initially I wanted to try to join the shapefile with the database file using the join field command. Once that didn't work out I referenced using Cursor to access the data from the ArcGIS website. I used Update cursor to join the two FIPS colums together in the shapefile. From that I was able to convert the county shapefiles into GeoJSON files. I used cursor to access the population data and then used that data to extract the ten most populated counties. The whole process took 10 hours to complete. 